<h1 style=\"text-align: center; font-size: 50px;\"> <h1 style=\"text-align: center; font-size: 50px;\"> 📦 Register Model </h1> </h1>

📘 Project Overview: 
 This notebook demonstrates a modular architecture for answering natural language questions 
 over one or more feedback documents using only local and open-source models (e.g., LLaMA.cpp).
 The system processes long documents chunk-by-chunk and synthesizes a final answer using a multi-step LLM workflow.

# Notebook Overview

- Start Execution
- Define User Constants
- Install and Import Libraries
- Configure Settings
- Verify Assets
- KV Memory
- LLM Setup
- State Model
- Node Functions
- Graph Definition
- Graph Visualization
- Generated Answer
- Message History

# Start Execution

In [1]:
# ─────── Standard Library Imports ───────
import os  # OS-level utilities like path and environment operations
import sys  # Access to interpreter variables and runtime configuration
import time  # Time-related functions

# Extend sys.path to include parent directory for local module resolution
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

# ─────── Local Application Imports ───────
from src.utils import (  # Core utilities for logging, LLM interaction, and schema generation
    display_image,
    get_response_from_llm,
    json_schema_from_type,
    log_timing,
    logger,
)

In [2]:
start_time = time.time()  
logger.info("Notebook execution started.")

# Define User Constants

In [3]:
TOPIC: str = "Focus Flow"  
QUESTION: str = "Which poeple provided the feedback?"

# Install and Import Libraries

In [4]:
%%time

%pip install -r ../requirements.txt --quiet

Note: you may need to restart the kernel to use updated packages.
CPU times: user 21.9 ms, sys: 8.51 ms, total: 30.4 ms
Wall time: 1.43 s


In [5]:
from __future__ import annotations  # Enables postponed evaluation of type annotations (PEP 563)

# ─────── Standard Library Imports ───────
import base64  # Encoding and decoding binary data
import functools  # Functional programming utilities like lru_cache, partial, etc.
import json  # JSON serialization and deserialization
import logging  # Logging framework
import multiprocessing  # Parallel execution using subprocesses
import os  # OS-level utilities
import shutil  # File and directory operations
import sys  # Access to runtime environment and system-specific parameters
import time  # Time tracking and delays
import warnings  # Warning control and filtering
from collections import namedtuple  # Lightweight object types
from pathlib import Path  # Object-oriented filesystem paths
from typing import Any, Dict, List, Literal, Optional, TypedDict  # Static typing annotations

# ─────── Third-Party Package Imports ───────
import mlflow  # Model tracking and serving framework
from mlflow.tracking import MlflowClient  # Interface to interact with MLflow tracking server for experiments, runs, and artifacts
import yaml  # YAML file parsing
from IPython.display import HTML, display, Markdown  # Rich output formatting in Jupyter environments
from tqdm import tqdm  # Progress bar for loops

# ─────── LangChain Core & Community Imports ───────
from langchain.docstore.document import Document  # Document abstraction
from langchain.text_splitter import RecursiveCharacterTextSplitter  # Intelligent text splitting
from langchain_community.document_loaders import (  # Document loaders for different file types
    CSVLoader,
    PyPDFLoader,
    TextLoader,
    UnstructuredExcelLoader,
    UnstructuredMarkdownLoader,
    UnstructuredWordDocumentLoader,
)
from langchain_community.llms import LlamaCpp  # Integration for running LlamaCpp locally

# ─────── LangGraph Imports ───────
from langgraph.graph import END, START, StateGraph  # Constructs and controls stateful agent graphs

# ─────── Local Application-Specific Imports ───────
from src.agentic_feedback_model import AgenticFeedbackModel  # Core agent logic for feedback analysis
from src.simple_kv_memory import SimpleKVMemory  # In-memory store for agent state

# Configure Settings

In [6]:
# Suppress Python warnings
warnings.filterwarnings("ignore")

In [7]:
INPUT_PATH: Path = Path("../data/input")  

MEMORY_PATH: Path = Path("../data/memory")  

CONFIG_PATH: Path = Path("../configs/config.yaml") 

MODEL_PATH = "/home/jovyan/datafabric/meta-llama3.1-8b-Q8/Meta-Llama-3.1-8B-Instruct-Q8_0.gguf"
CONTEXT_WINDOW = 8192
MAX_TOKENS = CONTEXT_WINDOW // 8
CHUNK_SIZE = CONTEXT_WINDOW // 2
CHUNK_OVERLAP = CHUNK_SIZE // 8  

EXPERIMENT_NAME = "AIStudio-Agentic-Customer-Feedback-Analyzer-with-LangGraph-Experiment"
RUN_NAME = "AIStudio-Agentic-Customer-Feedback-Analyzer-with-LangGraph-Run"
MODEL_NAME = "AIStudio-Agentic-Customer-Feedback-Analyzer-with-LangGraph-Model"

In [8]:
logger.info('Notebook execution started.')

## Verify Assets

In [9]:
def log_asset_status(asset_path: str, asset_name: str) -> None:
    """
    Logs the status of a given asset based on its existence.

    Parameters:
        asset_path (str): File or directory path to check.
        asset_name (str): Name of the asset for logging context.
    """
    if Path(asset_path).exists():
        logger.info(f"{asset_name} is properly configured.")
    else:
        logger.info(f"{asset_name} is not properly configured. Please ensure the required asset is correctly configured in your AI Studio project according to the README file.")

In [10]:
log_asset_status(
    asset_path=INPUT_PATH,
    asset_name="Input Data",
)
log_asset_status(
    asset_path=MODEL_PATH,
    asset_name="LLM",
)

# KV Memory

In [11]:
memory: SimpleKVMemory = SimpleKVMemory(MEMORY_PATH)
memory.set('dummy key', 'dummy value')

# Load Documents

In [12]:
logger.info("📂 Scanning directory for documents: %s", INPUT_PATH)

supported_extensions = {
".txt": TextLoader,
".csv": lambda path: CSVLoader(path, encoding="utf-8", csv_args={"delimiter": ","}),
".xlsx": UnstructuredExcelLoader,
".docx": UnstructuredWordDocumentLoader,
".pdf": PyPDFLoader,
".md": UnstructuredMarkdownLoader,
}

all_docs = []

for file_path in Path(INPUT_PATH).rglob("*"):
    # Skip hidden/system folders
    if any(part.startswith(".") and part not in {".", ".."} for part in file_path.parts):
        continue
    
    ext = file_path.suffix.lower()
    loader_class = supported_extensions.get(ext)
    
    if loader_class:
        try:
            loader = loader_class(str(file_path))
            docs = loader.load()
            all_docs.extend(docs)
            logger.info("✅ Loaded %d docs from %s", len(docs), file_path.name)
        except Exception as e:
            logger.warning("❌ Failed to load %s: %s", file_path.name, e)
    else:
        logger.info("⚠️ Unsupported file type: %s", file_path.name)

short text: "docx test". Defaulting to English.


short text: "md test". Defaulting to English.


short text: "excel test excel test". Defaulting to English.


In [13]:
INPUT_TEXT = '\n\n'.join([doc.page_content for doc in all_docs])

# MLflow Registration

In [14]:
# 1. Set MLflow tracking URI and experiment
mlflow.set_tracking_uri(os.getenv("MLFLOW_TRACKING_URI", "/phoenix/mlflow"))
mlflow.set_experiment(experiment_name=EXPERIMENT_NAME)
print(f"Using MLflow tracking URI: {mlflow.get_tracking_uri()}")
print(f"Experiment: {EXPERIMENT_NAME}")

Using MLflow tracking URI: /phoenix/mlflow
Experiment: AIStudio-Agentic-Customer-Feedback-Analyzer-with-LangGraph-Experiment


In [15]:
%%time

# These should point to the actual files you're using for model and memory
MODEL_ARTIFACTS = {
    "model_path": str(MODEL_PATH),
    "memory_path": str(MEMORY_PATH),
    "config_path": str(CONFIG_PATH),
    "demo": "../demo",
}
 
# === Start MLflow run, log, and register ===
with mlflow.start_run(run_name=RUN_NAME) as run:
    print(f"🚀 Started MLflow run: {run.info.run_id}")

    # Log and register the model using the classmethod
    AgenticFeedbackModel.log_model(
        model_name=MODEL_NAME,
        model_artifacts=MODEL_ARTIFACTS
    )

logger.info(f"✅ Model '{MODEL_NAME}' successfully logged and registered.")

2025/09/11 19:33:28 INFO mlflow.models.signature: Inferring model signature from type hints


🚀 Started MLflow run: b17845927951443fa391f61df512ca05


Registered model 'AIStudio-Agentic-Customer-Feedback-Analyzer-with-LangGraph-Model' already exists. Creating a new version of this model...
Created version '3' of model 'AIStudio-Agentic-Customer-Feedback-Analyzer-with-LangGraph-Model'.


CPU times: user 1.28 s, sys: 19.4 s, total: 20.7 s
Wall time: 3min 23s


In [16]:
# 3. Retrieve the latest version from the Model Registry
client = MlflowClient()
versions = client.get_latest_versions(MODEL_NAME, stages=["None"])

if not versions:
    raise RuntimeError(f"No registered versions found for model '{MODEL_NAME}'.")
    
latest_version = versions[0].version
model_info = mlflow.models.get_model_info(f"models:/{MODEL_NAME}/{latest_version}")

logger.info(f"Latest registered version of '{MODEL_NAME}': {latest_version}")
logger.info(f"Signature: {model_info.signature}")

In [ ]:
%%time

# 4. Load the model from the Model Registry
loaded_model = mlflow.pyfunc.load_model(model_uri=f"models:/{MODEL_NAME}/{latest_version}")
logger.info(f"Successfully loaded model '{MODEL_NAME}' version {latest_version} for inference.")

In [ ]:
# 5. Run a sample inference using the loaded model
input_payload = [{"topic": TOPIC, "question": QUESTION, "input_text": INPUT_TEXT, }]

print("\n=== Running Sample Inference ===")
results = loaded_model.predict(input_payload)
result = results[0]

# Generated Answer

In [ ]:
display(Markdown(result.answer))

# Message History

In [ ]:
print(result.messages)

In [ ]:
end_time: float = time.time()
elapsed_time: float = end_time - start_time
elapsed_minutes: int = int(elapsed_time // 60)
elapsed_seconds: float = elapsed_time % 60

logger.info(f"⏱️ Total execution time: {elapsed_minutes}m {elapsed_seconds:.2f}s")
logger.info("✅ Notebook execution completed successfully.")

Built with ❤️ using [**HP AI Studio**](https://hp.com/ai-studio).